1. training resnet18

In [203]:
import torch
import torchvision
import numpy as np
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import ToTensor,ToPILImage
from torchvision.io import read_image
from torchvision import models
from PIL import Image
import os
import io
import pandas as pd
import math
from torch.optim import Adam
from torchvision.models.resnet import BasicBlock

#os.environ["CUDA_VISIBLE_DEVICES"] = '5'

In [204]:
# Step 1, fine-tune VGG16 on CIFAR-10

transform = transforms.Compose([
   transforms.Resize(32),
   transforms.RandomCrop((32, 32), padding=4),
   transforms.RandomHorizontalFlip(p=0.5),
   ToTensor(),
   transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

transform_val = transforms.Compose([
   ToTensor(),
   transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])


batch_size = 100

trainset = torchvision.datasets.CIFAR10(root='../__data', train=True,
                                        download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../__data', train=False,
                                        download=True, transform=transform_val)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [205]:
train_loader = torch.utils.data.DataLoader(trainset
    , batch_size = batch_size
    , shuffle = True)
test_loader = torch.utils.data.DataLoader(testset
    , batch_size = batch_size
    , shuffle = False)
n_total_step = len(train_loader)
print(n_total_step)

500


In [206]:
!pip install torchmetrics

import random

def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)


import torch
from torchmetrics import MetricCollection
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall
metric_collection = MetricCollection([
    MulticlassAccuracy(num_classes=10, average="micro"),
    MulticlassPrecision(num_classes=10, average="macro"),
    MulticlassRecall(num_classes=10, average="macro")
])

def get_metrics(device, model, test_loader):
  with torch.no_grad():
    model.to(device)
    model.eval()
    for x, y in test_loader:
      x = x.to(device)
      y = y
      outputs = model(x)
      _, predicted = outputs.max(1)
      metric_collection.update(predicted.cpu(), y)

  metric_output = metric_collection.compute()

  #print(metric_output)

  acc, precision, recall = metric_output['MulticlassAccuracy'], metric_output['MulticlassPrecision'], metric_output['MulticlassRecall']
  metric_collection.reset()

  return acc, precision, recall


def get_metrics_decore(device, model, test_loader):
  with torch.no_grad():
    model.eval()
    for x, y in test_loader:
      x = x.to(device)
      y = y
      outputs,_ = model(x)
      _, predicted = outputs.max(1)
      metric_collection.update(predicted.cpu(), y)

  metric_output = metric_collection.compute()

  #print(metric_output)

  acc, precision, recall = metric_output['MulticlassAccuracy'], metric_output['MulticlassPrecision'], metric_output['MulticlassRecall']
  metric_collection.reset()

  return acc, precision, recall

In [207]:
import torch.optim as optim
from google.colab import drive
# import torch.optim as optim
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#resnet
model = models.resnet18(pretrained=True)

# Modify the first convolutional layer for CIFAR-10
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()  # Remove maxpool as CIFAR-10 images are smaller
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)

#print(model)

In [209]:
import time

# Examine VGG16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_epochs = 10
#batch_size = 40
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=1e-4)
#optimizer = Adam(model.parameters(), lr=learning_rate)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

best_acc = 0

cuda:0


In [ ]:
import time

start_time = time.time()
model.train()

for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        n_corrects = (outputs.argmax(axis=1)==labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 250 == 0:
            accuracy, precision, recall = get_metrics(device, model, test_loader)
            print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss:.5f}, acc = {100*(n_corrects/labels.size(0)):.2f}%, val_acc = {accuracy * 100:.2f}%')
            if best_acc < accuracy:
                best_acc = accuracy
                print(f'save best accuracy: {best_acc}')
                torch.save(model, './resnet_model_best.pth')

    # scheduler.step()

    print()

# 학습 종료 시간 기록
end_time = time.time()

# 총 학습 시간 계산
total_time = end_time - start_time
print(f"모델 학습 시간: {total_time // 60:.0f}분 {total_time % 60:.0f}초")

print("Resnet Training completed!!!")

accuracy, precision, recall = get_metrics(device, model, test_loader)

print ('[resnet] accuracy : ', accuracy, ', precision : ', precision, ', recall : ', recall)

# 전체 모델 저장
torch.save(model, '/content/drive/MyDrive/decore_model/resnet_model.pth')

2. Decore with VGG16 network

In [211]:
class CustomLoss(nn.Module):
    def __init__(self, reward, penalty):
        super().__init__()
        self.reward = reward
        self.penalty = penalty # How much to penalize incorrect drop

    def forward(self, agents_list, y_predicted, target):
        loss = 0.0
        N,_ = y_predicted.size()
        y_predicted = y_predicted.argmax(axis = 1)
        t = torch.Tensor(y_predicted.shape).to(y_predicted.device)
        t[y_predicted==target] = self.reward
        t[y_predicted!=target] = self.penalty
        # print(t)
        for a in agents_list:
            R = a.R*t
            loss += torch.mean(torch.sum(torch.log(a.Pi),dim=1)* R)
        return -loss # The aim is to maximize rewards

class Agent(nn.Module):
    def __init__(self, id, num_channels, on_off_rate = 1):
        print("agent made")
        super().__init__()
        self.id = id
        self.on_off_rate = on_off_rate
        self.num_channels = num_channels
        self.register_parameter(f'S_{id}',nn.Parameter(torch.ones(num_channels) * 6.9))
        _,S = next(self.named_parameters())
        p = nn.Sigmoid()(S.expand(1,num_channels))
        self.A = torch.bernoulli(p) # Actions
        self.Pi = torch.Tensor(self.A.shape) # policy probability
        self.Pi[self.A==1] = p[self.A==1]
        self.Pi[self.A==0] = 1 - p[self.A==0]
        self.R = torch.sum(abs(self.on_off_rate - self.A),dim=1) # Reward
        self.fixed_random = False

    def forward(self,I):
        #print("agent forward")
        N,c,h,w = I.shape
        _,S = next(self.named_parameters())

        p = nn.Sigmoid()(S.expand(N,c))
        if self.training and not self.fixed_random:
            self.A = torch.bernoulli(p)
        elif p.shape != self.A.shape:
            self.A = self.A.expand(p.shape).to(S.device)

        #self.A =self.A
        self.Pi = torch.Tensor(self.A.shape).to(S.device)
        self.Pi[self.A==1] = p[self.A==1]
        self.Pi[self.A==0] = 1 - p[self.A==0]
        self.R = torch.sum(abs(self.on_off_rate - self.A),dim=1)
        # print(f'agent[{self.id}] is forwarded..')
        return I*self.A.view(N,c,1,1).contiguous().expand(N,c,h,w)

    def fixed_random_seed(self, flag):
        self.fixed_random = flag

    def count_active_channels(self):
        """ 현재 활성화된 채널 수를 반환 """
        return torch.sum(self.A[-1]).item()

    def count_all_channels(self):
        """ 모든 채널 수를 반환 """
        return self.A[-1].numel()

    def __repr__(self):
        return f"Agent({self.id}, {self.num_channels})"

class DECORE_basicblock(nn.Module):
    def __init__(self, block, agent_id_offset): #in_num, out_num, conv1_stride, conv2_stride, downsample):
        super().__init__()
        self.agent_list = []
        #DECORE_basicblock(a[idx].conv1.in_channels, a[idx].conv2.out_channels, a[idx].conv1.stride, a[idx].conv2.stride, a[idx].downsample)
        self.conv1 = block.conv1
        self.bn1 = block.bn1
        self.relu = block.relu
        self.agent1 = Agent(agent_id_offset, block.conv1.out_channels)
        self.conv2 = block.conv2
        self.bn2 = block.bn2

        self.downsample = block.downsample
        # setting bypass conv for residual input
        # if self.downsample is None:
        #     self.conv_bypass = nn.Conv2d(block.conv1.in_channels, block.conv2.out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        #     # self.conv_bypass.weight.copy_(torch.eye(self.conv_bypass.in_channels).view(self.conv_bypass.out_channels, self.conv_bypass.in_channels, 1, 1))
        #     # self.conv_bypass = self.conv_bypass.weight.copy_(torch.eye(self.conv_bypass.in_channels).view(self.conv_bypass.out_channels, self.conv_bypass.in_channels, 1, 1))
        # else:
        #     self.conv_bypass = nn.Conv2d(self.downsample[0].out_channels, block.conv2.out_channels, kernel_size=1, stride=1, padding=0, bias=False)

        # with torch.no_grad():
        #     self.conv_bypass.weight.copy_(torch.eye(self.conv_bypass.in_channels).view(self.conv_bypass.out_channels, self.conv_bypass.in_channels, 1, 1))
        # # with torch.no_grad():
        # #     self.conv_bypass.weight.fill_(1.0)
        # self.conv_bypass.requires_grad = False

        # self.agent2 = Agent(agent_id_offset + 1, block.conv2.out_channels)
        self.agent_list.append(self.agent1)
        # self.agent_list.append(self.agent2)

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        # print('**********************************')
        # print(f'input x shape: {x.shape}')
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.agent1(out)
        # print(f'after conv1 out shape: {out.shape}')

        #out = self.agent1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        # out = self.agent2(out)
        # self.agent2.fixed_random_seed(True)
        # print(f'decore_conv2_agent_A: {self.agent2.A[0]}')

        # print(f'after conv2 out shape: {out.shape}')

        if self.downsample is not None:
            # print(f'before downsample x shape: {x.shape}')
            identity = self.downsample(x)
            # print(f'after downsample identity shape: {identity.shape}')

        # print(f'before conv_bypass identity shape: {identity.shape}')
        # identity = self.conv_bypass(identity)
        # print(f'after conv_bypass identity shape: {identity.shape}')
        # print(f'before agent_identity')
        # print(identity[0][0])
        # identity = self.conv_bypass(identity)
        # print(f'after agent_identity')
        # print(identity[0][0])
        # identity = self.agent2(identity)
        # self.agent2.fixed_random_seed(False)
        # print(f'conv_bypass weight')
        # print(self.conv_bypass.weight)
        # print(f'decore_indentity_agent_A: {self.agent2.A[0]}')

        out += identity
        out = self.relu(out)
        # print(f'last out shape: {out.shape}')

        return out

    def count_all_channels(self):
        return self.agent1.count_all_channels() + self.agent2.count_all_channels()


class DECOR_RESNET(nn.Module):
    def __init__(self, Model):
        super().__init__()
        import copy
        self.target_model = copy.deepcopy(Model)
        self.agents_list = [] # To keep track of all newly added agents
        self.block_list = []
        self.parse_model()
        self.agents = nn.ModuleList(self.agents_list)
        # self.blocks = nn.ModuleList(self.block_list)

    def parse_model(self):
        """ This method parse the target model and append agents after the activation function of each convolution, the dictionary of the
            target model should be defined for easy parsing """
        n = ''
        #modules = torch.nn.Sequential()

        for i in self.target_model.state_dict().keys():
            if n != i.split('.')[0]:
                n = i.split('.')[0]
                a = getattr(self.target_model, f'{n}')
                if hasattr(a, '__iter__'):
                    modules = torch.nn.Sequential()
                    for idx in range(0,len(a)):
                        from torchvision.models.resnet import BasicBlock
                        if isinstance(a[idx], BasicBlock):
                            block = DECORE_basicblock(a[idx], len(self.agents_list)) #DECORE_basicblock(a[idx].conv1.in_channels, a[idx].conv2.out_channels, a[idx].conv1.stride, a[idx].conv2.stride, a[idx].downsample)
                            modules.add_module(f'{idx}', block)
                            self.block_list.append(block)
                            for agnt in block.agent_list:
                                self.agents_list.append(agnt)

                    a = modules
                    setattr(self.target_model, f'{n}',a)

    def forward(self,I):
        return self.target_model(I)

    def frozen_agent(is_fronzen=True):
        for agent in self.agents_list:
            for name, param in agent.named_parameters():
                param.requires_grad = not is_fronzen

    def count_total_active_channels(self):
        """ 모든 에이전트에서 활성화된 채널 수의 총합을 반환 """
        total_active_channels = sum(agent.count_active_channels() for agent in self.agents)
        return total_active_channels

    def count_total_all_channels(self):
        """ 모든 에이전트에서 모든 채널 수의 총합을 반환 """
        #print('agent총 수 : ', len(self.agents))
        total_channels = sum(agent.count_all_channels() for agent in self.agents)
        return total_channels

    def print_active_channel(self):
        """각 에이전트의 활성화된 채널 수 출력"""
        for index, agent in enumerate(self.agents, 1):
            #print(f"Agent {index} 상태 - A: {agent.A[0]}")  # A 벡터 출력
            active_channels = agent.count_active_channels()
            print(f"CNN Layer #{index} Active Channels: {active_channels} / All Channels: {agent.count_all_channels()}")


In [212]:
#model = torch.load('./resnet18_bestacc.pth')
model = torch.load('/content/drive/MyDrive/decore_model/resnet_model.pth')
print(model)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
accuracy, precision, recall = get_metrics(device, model, test_loader)
model.train()
print ('[before resnet] decore accuracy : ', accuracy, ', precision : ', precision, ', recall : ', recall)

<ipython-input-212-1282e41fbed7>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/drive/MyDrive/decore_model/resnet_model.pth')


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [213]:
net = DECOR_RESNET(model)
net = net.to(device)
print(net)

accuracy, precision, recall = get_metrics(device, net, test_loader)
model.train()
print ('[before resnet] decore accuracy : ', accuracy, ', precision : ', precision, ', recall : ', recall)

agent made
agent made
agent made
agent made
agent made
agent made
agent made
agent made
DECOR_RESNET(
  (target_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): DECORE_basicblock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (agent1): Agent(0, 64)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): DECORE_basicblock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [214]:
id = 0
param = []

for n,p in net.named_parameters():
    if n.endswith(f".S_{id}") and p.requires_grad:
        param.append(p)
        id += 1

#optimizer = Adam(param, lr=lr)

# optimizer = optim.Adam(net.agents.parameters(), lr=lr) # 이거로 변경해보기
optimizer = optim.Adam(net.agents.parameters(), lr=0.01) # 이거로 변경해보기
#optimizer = torch.optim.SGD(net.agents.parameters(), lr = lr, momentum=0.9,weight_decay=5e-4)
# optimizer = torch.optim.SGD(net.parameters(), lr = lr, momentum=0.9, weight_decay=5e-4)
# optimizer = torch.optim.SGD(net.agents.parameters(), lr = lr)
#scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,40,60,80], gamma=0.5)
criterion_decore = CustomLoss(1, -30)
print(f'reward: {criterion_decore.reward}, penalty: {criterion_decore.penalty}')

base_lr = 0.00001
base_criterion = nn.CrossEntropyLoss()
# base_optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)
base_optimizer = optim.Adam(net.parameters(), lr=base_lr)

import gc
import time

torch.set_printoptions(edgeitems=torch.inf)
gc.collect()
torch.cuda.empty_cache()
best_acc = 0
best_comp_rate = 1
best_score = 0

reward: 1, penalty: -30


In [ ]:
#wandb 로깅
!pip install wandb
!wandb login
import wandb
import numpy as np
from datetime import datetime

num_epochs = 5

current_date = datetime.now().strftime("%Y-%m-%d")  # 현재 날짜를 기반으로 run_name 생성(형식: YYYY-MM-DD)
run_name = f"DECORE-PRE_RESNET-{current_date}"

wandb.init(
    project="DECORE-PRE_RESNET",
    name=run_name,
    config={
        "architecture": "ResNet18",
        "dataset": "CIFAR10",
        "epochs": num_epochs,
        "batch_size": batch_size,
        "learning_rate": base_lr,
        "reward": criterion_decore.reward,
        "penalty": criterion_decore.penalty
    }
)

def log_layer_metrics(net, epoch):
    """Log compression rates for each layer"""
    for idx, agent in enumerate(net.agents, 1):
        active_channels = agent.count_active_channels()
        total_channels = agent.count_all_channels()
        compression_rate = active_channels / total_channels

        # Add row to the table
        layer_compression_table.add_data(epoch, f"Layer {idx}", active_channels, total_channels, compression_rate)

        # Log individual layer metrics
        wandb.log({
            f"layer_{idx}/compression_rate": compression_rate,
            f"layer_{idx}/active_channels": active_channels
        })

def log_training_metrics(epoch, step, total_loss, running_corrects, total_samples,
                        test_accuracy, test_precision, test_recall, total_active_channel,
                        total_all_channel):
    """Log training and testing metrics"""
    train_accuracy = running_corrects / total_samples
    compression_rate = total_active_channel / total_all_channel

    wandb.log({
        "epoch": epoch,
        "step": step,
        "loss": total_loss,
        "train_accuracy": train_accuracy,
        "test_accuracy": test_accuracy,
        "test_precision": test_precision,
        "test_recall": test_recall,
        "overall_compression_rate": compression_rate,
        "active_channels": total_active_channel,
        "total_channels": total_all_channel
    })

In [216]:
print("decore training start")
# 학습 시작 시간 기록
start_time = time.time()
net.train()

# wandb table 만들기
epoch_table = wandb.Table(columns=["epoch", "train_loss", "train_accuracy", "test_accuracy",
                                 "compression_rate", "active_channels", "total_channels"])

metrics_table = wandb.Table(
    columns=["epoch", "train_loss", "train_accuracy", "test_accuracy",
             "compression_rate", "active_channels", "total_channels"])

layer_table = wandb.Table(
    columns=["epoch"] + [f"layer_{i}_compression" for i in range(len(net.agents_list))])

for epoch in range(num_epochs):
    running_corrects = 0 # Accuracy calculation
    total_samples = 0 # Accuracy calculation
    total_loss = 0
    total_all_channel = 0
    total_active_channel = 0

    # epoch별
    epoch_total_loss = 0
    epoch_running_corrects = 0
    epoch_total_samples = 0
    epoch_all_channel = 0
    epoch_active_channel = 0

    #train_loss = 0.0
    for i, (imgs , labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        labels_hat = net.target_model(imgs)
        loss_value = criterion_decore(net.agents_list, labels_hat, labels)

        _, preds = torch.max(labels_hat, 1)
        running_corrects += torch.sum(preds == labels).item()
        total_samples += labels.size(0)

        total_loss += loss_value
        total_all_channel += net.count_total_all_channels()
        total_active_channel += net.count_total_active_channels()
        curr_comp_rate = total_active_channel / total_all_channel

        # epoch별
        epoch_running_corrects += torch.sum(preds == labels).item()
        epoch_total_samples += labels.size(0)
        epoch_total_loss += loss_value.item()
        epoch_all_channel += net.count_total_all_channels()
        epoch_active_channel += net.count_total_active_channels()

        if (i + 1) % (n_total_step / 2) == 0:
            accuracy, precision, recall = get_metrics(device, net, test_loader)
            # print ('decore accuracy : ', accuracy, ', precision : ', precision, ', recall : ', recall)
            net.train()

            print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {total_loss:.5f}, train_acc = {(100 * running_corrects / total_samples):.2f}%, test_acc = {accuracy * 100:.2f}%, channel = { curr_comp_rate * 100:.2f}%')
            # print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {total_loss:.5f}, train_acc = {(100 * running_corrects / total_samples):.2f}%, channel = { total_active_channel / total_all_channel * 100:.2f}%, lr ={optimizer.param_groups[0]["lr"]}')
            active_cnt_list = []
            for agnt in net.agents_list:
                active_cnt_list.append(int(agnt.count_active_channels()))
            print(active_cnt_list)
            if 0:
                for agent in net.agents_list:
                    rounded_Pi = torch.round(agent.Pi * 10000) / 10000
                    unique_values, counts = torch.unique(rounded_Pi, return_counts=True)
                    #print(agent.Pi[agent.Pi != 0.999])
                    for value, count in zip(unique_values, counts):
                        print(f"Value: {value.item():.5f}, Count: {count.item()}")
                    break
                total_loss = 0

        #train_loss += loss_value.detach().cpu().item() / len(train_loader)
        optimizer.zero_grad()
        #loss_value = loss_value.detach()
        loss_value.backward()
        optimizer.step()
        #scheduler.step()


    # epoch 종료 후 metrics 계산 및 기록
    epoch_avg_loss = epoch_total_loss / len(train_loader)
    epoch_accuracy = epoch_running_corrects / epoch_total_samples
    epoch_comp_rate = epoch_active_channel / epoch_all_channel

    # 테스트 metrics 계산
    final_accuracy, final_precision, final_recall = get_metrics(device, net, test_loader)
    net.train()

    # epoch table에 데이터 추가
    metrics_table.add_data(
        epoch + 1,
        float(epoch_avg_loss),  # tensor를 float로 변환
        float(epoch_accuracy * 100),
        float(final_accuracy * 100),
        float(epoch_comp_rate * 100),
        int(epoch_active_channel),
        int(epoch_all_channel)
    )

    # 레이어별 압축률 계산 및 기록
    layer_compressions = [epoch + 1]
    for agent in net.agents_list:
        active_channels = agent.count_active_channels()
        total_channels = agent.Pi.shape[0]
        compression_rate = (active_channels / total_channels) * 100
        layer_compressions.append(compression_rate)
    layer_compression_table.add_data(*layer_compressions)

    # wandb에 epoch 단위 metrics 기록
    wandb.log({
        "epoch": epoch + 1,
        "train/loss": epoch_avg_loss,
        "train/accuracy": epoch_accuracy * 100,
        "test/accuracy": final_accuracy * 100,
        "test/precision": final_precision * 100,
        "test/recall": final_recall * 100,
        "compression/rate": epoch_comp_rate * 100,
        "compression/active_channels": epoch_active_channel,
        "compression/total_channels": epoch_all_channel
    })

    if 1: # fine-tuning
        if (epoch + 1) % 5 == 0:
            # base model training
            for agent in net.agents_list:
                for name, param in agent.named_parameters():
                    param.requires_grad = False


            ft_metrics = {
                "loss": [],
                "train_acc": [],
                "test_acc": [],
                "score": []
            }

            for ft_epoch in range(5):
                ft_running_loss = 0
                ft_corrects = 0
                ft_total = 0

                for i, (imgs , labels) in enumerate(train_loader):
                    imgs = imgs.to(device)
                    labels = labels.to(device)

                    labels_hat = net.target_model(imgs)
                    n_corrects = (labels_hat.argmax(axis=1)==labels).sum().item()
                    ft_corrects += n_corrects
                    ft_total += labels.size(0)

                    base_loss_value = base_criterion(labels_hat, labels)
                    base_loss_value.backward()
                    base_optimizer.step()
                    base_optimizer.zero_grad()
                    ft_running_loss += base_loss_value.item()

                    if (i+1) % (n_total_step / 2) == 0:
                        accuracy, precision, recall = get_metrics(device, net, test_loader)
                        net.train()
                        score = accuracy / curr_comp_rate
                        print(f'[base_model] epoch {epoch+1}/{5}, step: {i+1}/{n_total_step}: loss = {base_loss_value:.5f}, acc = {100*(n_corrects/labels.size(0)):.2f}%, test_acc = {accuracy * 100:.2f}%, score = {score}')
                        if best_score < score:
                            best_score = score
                            print(f'save best score model: {best_score:.4f} (comp rate: {curr_comp_rate:.4f}, acc: {accuracy:.4f})')
                            torch.save(net, f'./decore_resnet_best.pth')

                ft_epoch_loss = ft_running_loss / len(train_loader)
                ft_train_acc = ft_corrects / ft_total
                ft_test_acc, _, _ = get_metrics(device, net, test_loader)
                ft_score = ft_test_acc / curr_comp_rate

                # wandb에 fine-tuning metrics 기록
                wandb.log({
                    "epoch": epoch + 1,
                    "finetune/epoch": ft_epoch + 1,
                    "finetune/loss": ft_epoch_loss,
                    "finetune/train_accuracy": ft_train_acc * 100,
                    "finetune/test_accuracy": ft_test_acc * 100,
                    "finetune/score": ft_score
                })

            for agent in net.agents_list:
                for name, param in agent.named_parameters():
                    param.requires_grad = True

    if epoch % 100 == 0:
        # Google Drive에 모델 가중치 저장
        torch.save(net.state_dict(), f'./decore_resnet_model_weights_{epoch}.pth')
        # 전체 모델 저장
        torch.save(net, f'./decore_resnet_model_{epoch}.pth')

# wandb table 기록
wandb.log({
    "epoch_metrics": epoch_table,
    "layer_compression_history": layer_compression_table
})

# wandb end
wandb.finish()

# 학습 종료 시간 기록
end_time = time.time()

# 총 학습 시간 계산
total_time = end_time - start_time
print(f"decore 모델 학습 시간: {total_time // 60:.0f}분 {total_time % 60:.0f}초")
print("총 채널 수 : ", net.count_total_all_channels())
print("활성화된 채널 수 : ",net.count_total_active_channels())

net.eval()
accuracy, precision, recall = get_metrics(device, net.target_model, test_loader)

print ('decore accuracy : ', accuracy, ', precision : ', precision, ', recall : ', recall)

# DECORE 모델에서 활성화된 채널 정보 가져오기
active_channels_list = []
for agent in net.agents:
    agent.eval()
    # 각 agent에서 활성화된 채널의 인덱스 가져오기 (1차원으로 추출)
    active_channels_list.append(agent.A[-1].detach().cpu().numpy())

for i, active_channels in enumerate(active_channels_list):
    print(f"Layer {i} active channels: {active_channels} / {active_channels.shape}")

# Google Drive에 모델 가중치 저장
torch.save(net.state_dict(), './decore_resnet_model_final_weights.pth')

# 전체 모델 저장
torch.save(net, './decore_resnet_model_final.pth')


# 전체 모델 저장
torch.save(net, '/content/drive/MyDrive/decore_model/decore_resnet_model.pth')


decore training start
epoch 1/5, step: 250/500: loss = -42999.37891, train_acc = 70.89%, test_acc = 72.14%, channel = 99.88%
[64, 64, 128, 128, 256, 254, 511, 511]
epoch 1/5, step: 500/500: loss = -77882.78906, train_acc = 70.97%, test_acc = 72.29%, channel = 99.90%
[64, 64, 128, 128, 256, 256, 511, 512]
epoch 2/5, step: 250/500: loss = -27537.19336, train_acc = 71.06%, test_acc = 72.55%, channel = 99.92%
[64, 64, 128, 128, 256, 255, 512, 512]
epoch 2/5, step: 500/500: loss = -49249.40625, train_acc = 71.21%, test_acc = 72.35%, channel = 99.93%
[64, 64, 128, 128, 256, 255, 512, 512]
epoch 3/5, step: 250/500: loss = -18032.25586, train_acc = 71.22%, test_acc = 72.01%, channel = 99.95%
[64, 64, 128, 128, 256, 256, 512, 512]
epoch 3/5, step: 500/500: loss = -33048.63281, train_acc = 71.21%, test_acc = 72.17%, channel = 99.95%
[64, 64, 128, 128, 255, 256, 511, 511]
epoch 4/5, step: 250/500: loss = -13285.19336, train_acc = 71.08%, test_acc = 72.24%, channel = 99.96%
[64, 64, 128, 128, 256,

compression/active_channels,▁▄▆▇█
compression/rate,▁▄▆▇█
compression/total_channels,▁▁▁▁▁
epoch,▁▃▅▆██████
finetune/epoch,▁▃▅▆█
finetune/loss,█▅▃▂▁
finetune/score,▁▃▆▇█
finetune/test_accuracy,▁▃▆▇█
finetune/train_accuracy,▁▄▆▇█
test/accuracy,▇█▄▆▁
test/precision,▆▆▃█▁


decore 모델 학습 시간: 14분 15초
총 채널 수 :  1920
활성화된 채널 수 :  1920.0
decore accuracy :  tensor(0.8606) , precision :  tensor(0.8600) , recall :  tensor(0.8606)
Layer 0 active channels: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] / (64,)
Layer 1 active channels: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] / (64,)
Layer 2 active channels: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 

3. structured pruning

In [217]:
class NewBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None, index =0):
        super(NewBasicBlock, self).__init__()
        # 첫 번째 Conv 레이어
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        # 두 번째 Conv 레이어
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # downsample이 필요한 경우 (입력과 출력의 크기가 다를 경우 처리)
        self.downsample = downsample
        self.stride = stride
        self.index = index

        if self.downsample is None:
            self.conv_bypass = nn.Conv2d(self.conv1.in_channels, self.conv2.out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        else:
            self.conv_bypass = nn.Conv2d(self.downsample[0].out_channels, self.conv2.out_channels, kernel_size=1, stride=1, padding=0, bias=False)

        with torch.no_grad():
            in_channels = self.conv_bypass.in_channels
            out_channels = self.conv_bypass.out_channels
            # Initialize weight with zeros
            weight = torch.zeros(out_channels, in_channels, 1, 1)
            # Fill the matching channels with an identity matrix
            idx = min(in_channels, out_channels)
            weight[:idx, :idx, 0, 0] = torch.eye(idx)
            self.conv_bypass.weight.copy_(weight)
        self.conv_bypass.requires_grad = False

        # 디버깅을 위한 초기화 상태 저장
        self._debug_info = {
            "conv_bypass_weight": self.conv_bypass.weight.clone(),
        }

    def forward(self, x):
        #print("layer index : ", self.index)
        identity = x
        #agent conv의 inchannel수
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # downsample 경로를 통해 크기를 맞춤
        if self.downsample is not None:
            #print("down sample is not none")
            identity = self.downsample(x)

        # Skip connection
        identity = self.conv_bypass(identity)
        out += identity
        out = self.relu(out)

        return out

    def debug(self):
      """디버깅 정보를 출력하는 함수"""
      print(f"Debugging Block {self.index}:")

      # 초기화된 weight와 현재 weight의 차이 계산
      diff = self.conv_bypass.weight - self._debug_info["conv_bypass_weight"]
      nonzero_diff = diff.abs() > 1e-8  # 차이가 있는 위치 확인

      # 차이가 있는 요소 출력
      if torch.any(nonzero_diff):
          print("Bypass weight 차이 (non-zero elements):")
          diff_indices = nonzero_diff.nonzero(as_tuple=True)  # 차이가 있는 위치 인덱스
          for i in range(len(diff_indices[0])):
              print(f"Index {tuple(d[i] for d in diff_indices)}: {diff[diff_indices][i].item()}")
          print("오류남 ==> Bypass weights 바뀜")
      else:
          print("정상임 ==> Bypass weights 일정함")


class ModifiedResNet(nn.Module):
    def __init__(self, resnet18, active_channels_list):
        super(ModifiedResNet, self).__init__()
        self.features = nn.ModuleList()
        self.active_channels_list = active_channels_list
        prev_out_channels = 3
        conv_layer_count = 0

        for i, layer in enumerate(resnet18.children()):
            if isinstance(layer, nn.Sequential):
                new_seq_layers = []
                for j, sub_layer in enumerate(layer):
                    if isinstance(sub_layer, DECORE_basicblock):
                        active_out_channels = torch.tensor(self.active_channels_list[conv_layer_count], dtype=torch.bool)
                        out_indices = active_out_channels.nonzero().squeeze(1)

                        if conv_layer_count > 0:
                            active_in_channels = torch.tensor(self.active_channels_list[conv_layer_count - 1], dtype=torch.bool)
                            in_indices = active_in_channels.nonzero().squeeze(1)
                        else:
                            active_in_channels = torch.ones(prev_out_channels, dtype=torch.bool)
                            in_indices = active_in_channels.nonzero().squeeze(1)

                        # Downsample 경로 수정
                        if sub_layer.downsample is not None:
                            downsample = nn.Sequential()
                            before_conv1 = list(sub_layer.downsample.children())[0]
                            before_bn1 = list(sub_layer.downsample.children())[1]

                            # Downsample conv는 원본 채널 수 유지
                            conv1 = before_conv1
                            downsample.add_module('0', conv1)
                            downsample.add_module('1', before_bn1)
                        else:
                            downsample = None

                        # Create new block with original channel dimensions
                        new_block = NewBasicBlock(sub_layer.conv1.in_channels,
                                                sub_layer.conv2.out_channels,
                                                stride=sub_layer.conv1.stride,
                                                downsample=downsample,
                                                index=j)
                        new_block.debug()

                        # Conv1 수정 (출력 채널만 pruning)
                        if sub_layer.conv1.bias is None:
                            bias_flag = False
                        else:
                            bias_flag = True

                        conv1 = nn.Conv2d(sub_layer.conv1.in_channels,
                                        len(out_indices),
                                        kernel_size=sub_layer.conv1.kernel_size,
                                        stride=sub_layer.conv1.stride,
                                        padding=sub_layer.conv1.padding,
                                        bias=bias_flag)

                        copied_weights = sub_layer.conv1.weight.data[out_indices]
                        conv1.weight.data.copy_(copied_weights)
                        if bias_flag:
                            conv1.bias.data.copy_(sub_layer.conv1.bias.data[out_indices])
                        new_block.conv1 = conv1

                        # BN1 수정
                        bn1 = nn.BatchNorm2d(len(out_indices),
                                           sub_layer.bn1.eps,
                                           sub_layer.bn1.momentum,
                                           sub_layer.bn1.affine,
                                           sub_layer.bn1.track_running_stats)

                        bn1.weight.data.copy_(sub_layer.bn1.weight.data[out_indices])
                        bn1.bias.data.copy_(sub_layer.bn1.bias.data[out_indices])
                        if sub_layer.bn1.track_running_stats:
                            bn1.running_mean.copy_(sub_layer.bn1.running_mean[out_indices])
                            bn1.running_var.copy_(sub_layer.bn1.running_var[out_indices])
                        new_block.bn1 = bn1

                        # Conv2 수정 (입력 채널만 pruning)
                        if sub_layer.conv2.bias is None:
                            bias_flag = False
                        else:
                            bias_flag = True

                        conv2 = nn.Conv2d(len(out_indices),
                                        sub_layer.conv2.out_channels,
                                        kernel_size=sub_layer.conv2.kernel_size,
                                        stride=sub_layer.conv2.stride,
                                        padding=sub_layer.conv2.padding,
                                        bias=bias_flag)

                        copied_weights = sub_layer.conv2.weight.data[:, out_indices]
                        conv2.weight.data.copy_(copied_weights)
                        if bias_flag:
                            conv2.bias.data.copy_(sub_layer.conv2.bias.data)
                        new_block.conv2 = conv2

                        # bn2는 원본 그대로 사용
                        new_block.bn2 = sub_layer.bn2

                        conv_layer_count += 1
                        new_seq_layers.append(new_block)
                        prev_out_channels = sub_layer.conv2.out_channels
                    else:
                        new_seq_layers.append(sub_layer)

                self.features.append(nn.Sequential(*new_seq_layers))
            else:
                if isinstance(layer, nn.Conv2d):
                    prev_out_channels = layer.out_channels
                elif isinstance(layer, nn.Linear):
                    new_linear = layer  # 원본 linear 레이어 사용
                self.features.append(layer)
            if i == 9:
                break

    def forward(self, x):
        for layer in self.features:
            x = layer(x)
            if isinstance(layer, nn.AdaptiveAvgPool2d):
                x = torch.flatten(x, 1)
        return x


In [218]:
#print(net)

In [219]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# net = torch.load('./decore_resnet_best.pth')
# net = net.to(device)
# print(net)

accuracy, precision, recall = get_metrics(device, net, test_loader)
net.train()
print ('[before resnet decore] decore accuracy : ', accuracy, ', precision : ', precision, ', recall : ', recall)

[before resnet decore] decore accuracy :  tensor(0.8606) , precision :  tensor(0.8600) , recall :  tensor(0.8606)


In [220]:
# DECORE 모델에서 활성화된 채널 정보 가져오기
active_channels_list = []
for agent in net.agents:
    # 각 agent에서 활성화된 채널의 인덱스 가져오기 (1차원으로 추출)
    active_channels_list.append(agent.A[-1].detach().cpu().numpy())

for i, active_channels in enumerate(active_channels_list):
    print(f"Layer {i} active channels: {active_channels.sum().item()} / {active_channels.shape}")


modified_resnet = ModifiedResNet(net.target_model.eval(), active_channels_list).to(device)
print(modified_resnet)

accuracy, precision, recall = get_metrics(device, modified_resnet.eval(), test_loader)
print ('[modified resnet decore] accuracy : ', accuracy, ', precision : ', precision, ', recall : ', recall)


Layer 0 active channels: 64.0 / (64,)
Layer 1 active channels: 64.0 / (64,)
Layer 2 active channels: 128.0 / (128,)
Layer 3 active channels: 128.0 / (128,)
Layer 4 active channels: 256.0 / (256,)
Layer 5 active channels: 256.0 / (256,)
Layer 6 active channels: 512.0 / (512,)
Layer 7 active channels: 512.0 / (512,)
Debugging Block 0:
정상임 ==> Bypass weights 일정함
Debugging Block 1:
정상임 ==> Bypass weights 일정함
Debugging Block 0:
정상임 ==> Bypass weights 일정함
Debugging Block 1:
정상임 ==> Bypass weights 일정함
Debugging Block 0:
정상임 ==> Bypass weights 일정함
Debugging Block 1:
정상임 ==> Bypass weights 일정함
Debugging Block 0:
정상임 ==> Bypass weights 일정함
Debugging Block 1:
정상임 ==> Bypass weights 일정함
ModifiedResNet(
  (features): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Identity()
    (4): Sequential(
      (0): NewBasicBlock(
    